<a href="https://colab.research.google.com/github/Ashish-17CSE/Hate_Speech_Detection/blob/main/Hate_Speech_Detection.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Upload Data


In [ ]:
# Upload the train.csv and val.csv file from your local drive
from google.colab import files
uploaded = files.upload()

Saving dev_3k_split_conll.txt to dev_3k_split_conll (1).txt
Saving Hindi_test_unalbelled_conll_updated.txt to Hindi_test_unalbelled_conll_updated (1).txt
Saving test_labels_hinglish.txt to test_labels_hinglish (1).txt
Saving train_14k_split_conll .txt to train_14k_split_conll  (1).txt


# Preprocessing

In [ ]:
#libraries
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import re

import tensorflow as tf
import tensorflow.keras.preprocessing.text as kpt
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.layers import Embedding
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.preprocessing.text import one_hot
from tensorflow.keras.layers import LSTM
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import Bidirectional
from tensorflow.keras.layers import Dropout


import nltk
import json
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from nltk.tokenize import word_tokenize


In [ ]:
#!pip install turicreate
# !pip uninstall -y mxnet
# !pip install mxnet-cu100==1.4.0.post0

In [ ]:
# load datasets
train = pd.read_txt('/content/train_14k_split_conll.txt').fillna(' ')
# train.columns = ['Post', 'Labels Set']
# valid = pd.read_txt('/content/test_labels_hinglish.txt').fillna(' ')
train.head()
# valid.head(6)

AttributeError: ignored

In [ ]:
## Get the Independent Features
X=train.drop('Labels Set',axis = 1)

## Get the Dependent features
y=train['Labels Set']

KeyError: ignored

In [ ]:
y.value_counts(",")

non-hostile                       3050
fake                              1009
hate                               478
offensive                          405
defamation                         305
hate,offensive                     163
defamation,offensive                81
defamation,hate                     74
defamation,fake                     34
defamation,hate,offensive           28
fake,offensive                      28
fake,hate                           27
defamation,fake,offensive           24
defamation,fake,hate,offensive       9
defamation,fake,hate                 9
fake,hate,offensive                  4
Name: Labels Set, dtype: int64

In [ ]:
# Create a list of punctuation marks
puncts = [',', '.', '"', ':', ')', '(', '-', '!', '?', '|', ';', "'", '$', '&', '/', '[', ']', '>', '%', '=', '#', '*', '+', '\\', '•',  '~', '@', '£', 
 '·', '_', '{', '}', '©', '^', '®', '`',  '<', '→', '°', '€', '™', '›',  '♥', '←', '×', '§', '″', '′', 'Â', '█', '½', 'à', '…', 
 '“', '★', '”', '–', '●', 'â', '►', '−', '¢', '²', '¬', '░', '¶', '↑', '±', '¿', '▾', '═', '¦', '║', '―', '¥', '▓', '—', '‹', '─', 
 '▒', '：', '¼', '⊕', '▼', '▪', '†', '■', '’', '▀', '¨', '▄', '♫', '☆', 'é', '¯', '♦', '¤', '▲', 'è', '¸', '¾', 'Ã', '⋅', '‘', '∞', 
 '∙', '）', '↓', '、', '│', '（', '»', '，', '♪', '╩', '╚', '³', '・', '╦', '╣', '╔', '╗', '▬', '❤', 'ï', 'Ø', '¹', '≤', '‡', '√','#']

# Code to replace punctuations with whitespaces
def clean_text(x):
    x = str(x)
    for punct in puncts:
        if punct in x:
            x = x.replace(punct, ' ')
    return x

# Cleaning URLs, twitter user_handles, punctuations, whitespaces and converting to lowercase
# Training  Data
train['Post'] = train['Post'].apply(lambda x: re.sub(r'http\S+', '', x))
train['Post'] = train['Post'].apply(lambda x: re.sub("([^0-9A-Za-z \t])|(\w+:\/\/\S+)", '', x))
train['Post'] = train['Post'].apply(lambda x: clean_text(x))
train['Post'] = train['Post'].str.lower()
train['Post'] = train['Post'].apply(lambda x:' '.join(x.split()))

print(train.head())

# Validation  Data
valid['Post'] = valid['Post'].apply(lambda x: re.sub(r'http\S+', '', x))
valid['Post'] = valid['Post'].apply(lambda x: re.sub("([^0-9A-Za-z \t])|(\w+:\/\/\S+)*", '', x))
valid['Post'] = valid['Post'].apply(lambda x: clean_text(x))
valid['Post'] = valid['Post'].str.lower()
valid['Post'] = valid['Post'].apply(lambda x:' '.join(x.split()))


print(valid.head())

Onehot Representaion

In [ ]:
messages=X.copy()

In [ ]:

messages['Post']

In [ ]:
messages.reset_index(inplace=True)

In [ ]:
import nltk
import re
from nltk.corpus import stopwords

In [ ]:
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [ ]:
### Dataset Preprocessing
from nltk.stem.porter import PorterStemmer
ps = PorterStemmer()
corpus = []
for i in range(0, len(messages)):
    print(i)
    review = re.sub('[^a-zA-Z]', ' ', messages['Post'][i])
    review = review.lower()
    review = review.split()
    
    review = [ps.stem(word) for word in review if not word in stopwords.words('english')]
    review = ' '.join(review)
    corpus.append(review)

Embedding Representation

In [ ]:
corpus

In [ ]:
voc_size=6419

In [ ]:
onehot_repr=[one_hot(words,voc_size)for words in corpus] 
onehot_repr

In [ ]:
sent_length=20
embedded_docs=pad_sequences(onehot_repr,padding='pre',maxlen=sent_length)
print(embedded_docs)

In [ ]:
embedded_docs[0]

# Train the Model


1. Only use "train" data to train your model. For validation split the "train" data. 
2. Use whole "valid" for testing purpose. 

In [ ]:
def ConvolutionalNeuralNet(embeddings, max_sequence_length, num_words, embedding_dim, labels_index):
    
    # Embedding layer
    embedding_layer = Embedding(num_words,
                            embedding_dim,
                            weights=[embeddings],
                            input_length=max_sequence_length,
                            trainable=False)
    
    sequence_input = Input(shape=(max_sequence_length,), dtype='int32')
    embedded_sequences = embedding_layer(sequence_input)

    convs = []
    
    #Kernal (window) size
    filter_sizes = [2,3,4,5]

    # Convolution Layer and max-pooling
    for filter_size in filter_sizes:
        l_conv = Conv1D(filters=512, kernel_size=filter_size, activation='relu')(embedded_sequences)
        l_pool = GlobalMaxPooling1D()(l_conv)
        convs.append(l_pool)
        
    l_merge = concatenate(convs, axis=1)
    
    # softmax layer for multiclass identification 
    x = Dropout(0.1)(l_merge)  
    x = Dense(512, activation='relu')(x)
    x = Dropout(0.2)(x)
    preds = Dense(labels_index, activation='softmax')(x)

    model = Model(sequence_input, preds)
    model.compile(loss= 'categorical_crossentropy',
                  optimizer='adam',
                  metrics=['accuracy'])
   
    model.summary()
    return model

In [ ]:
label_names = []

In [ ]:
## Creating model
embedding_vector_features=40
model1=Sequential()
model1.add(Embedding(voc_size,embedding_vector_features,input_length=sent_length))
model1.add(Bidirectional(LSTM(100)))
model1.add(Dropout(0.3))
model1.add(Dense(1,activation='sigmoid'))
model1.compile(loss='binary_crossentropy',optimizer='adam',metrics=['accuracy'])
print(model1.summary())

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (None, 20, 40)            256760    
_________________________________________________________________
bidirectional (Bidirectional (None, 200)               112800    
_________________________________________________________________
dropout (Dropout)            (None, 200)               0         
_________________________________________________________________
dense_1 (Dense)              (None, 1)                 201       
Total params: 369,761
Trainable params: 369,761
Non-trainable params: 0
_________________________________________________________________
None


In [ ]:
len(embedded_docs),y.shape

(6420, (6420,))

In [ ]:
import numpy as np
X_final=np.array(embedded_docs)
y_final=np.array(y)

In [ ]:
X_final.shape,y_final.shape

((6420, 20), (6420,))

In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X_final, y_final, test_size=0.33, random_state=42)

Model Training

In [ ]:
### Finally Training
model1.fit(X_train,y_train,validation_data=(X_test,y_test),epochs=10,batch_size=64)

UnimplementedError: ignored

# Test the Model

# New Section

1. Save your weights with appropriate name
2. Test the model on "valid" dataset
3. print Confusion matrix and accuracy

In [ ]:
# coding begins again

# Driver Code for any input

1. Write a function which takes a single tweet and return fake or real
2. In the given function, first preprocess the given tweet, and than predict based on your model and return label. 

In [ ]:
# Final round of coding goes here